# STEP 5: ETL the data from 3NF tables to Facts & Dimension Tables
**IMPORTANT:** The following exercise depends on first having successing completed Exercise 1: Step 4. 

Start by running the code in the cell below to connect to the database. If you are coming back to this exercise, then uncomment and run the first cell to recreate the database. If you recently completed steps 1 through 4, then skip to the second cell.

In [ ]:
%%capture 
# ^hide output of this cell (comment to debug)

# run step 4 to create and populate tables
%run "E1 - Step 4.ipynb"

# retrieve tables from step 4
%store -r tables 

from src.database import (
    get_pg_connection,
    execute_pg_query,
    create_pg_table,
    insert_pg_rows,
    drop_pg_table,
    close_pg_connection,
)
from IPython.display import display
import pandas as pd

# get active connection
conn = get_pg_connection(sample_pagila=True)

### Introducing SQL to SQL ETL
When writing SQL to SQL ETL, you first create a table then use the INSERT and SELECT statements together to populate the table. Here's a simple example.

First, you create a table called test_table.

In [ ]:
table_name = "test_table"
drop_pg_table(conn, table_name)  # Drop if exists (nice for re-runs)
columns = {
    "date": "timestamp",
    "revenue": "decimal(5,2)",
}
create_pg_table(conn, table_name, columns)
tables[table_name] = columns

Then you use the INSERT and SELECT statements to populate the table. In this case, the SELECT statement extracts data from the `payment` table and INSERTs it INTO the `test_table`.

In [ ]:
rows = execute_pg_query(
    conn,
    """
    SELECT
        payment_date,
        amount
    FROM payment
    """,
    return_rows=True,
)
insert_pg_rows(conn, "test_table", tables["test_table"].keys(), rows)

Then you can use a SELECT statement to take a look at your new table.

In [ ]:
result = execute_pg_query(
    conn,
    """SELECT * FROM test_table LIMIT 5;
    """,
)
display(result.style.set_caption("First 5 rows of test_table").hide(axis="index"))

Great! Now you'll do the same thing below to create the dimension and fact tables for the Star Schema using the data in the 3NF database.

## ETL from 3NF to Star Schema

### 3NF - Entity Relationship Diagram

<img src="./../images/pagila-3nf.png" width="50%"/>

### Star Schema - Entity Relationship Diagram

<img src="./../images/pagila-star.png" width="50%"/>

Normalized tables

In [ ]:
from pprint import pprint
pprint(tables)

In this section, you'll populate the tables in the Star schema. You'll `extract` data from the normalized database, `transform` it, and `load` it into the new tables. 

To serve as an example, below is the query that populates the `dimDate` table with data from the `payment` table.
* NOTE 1: The EXTRACT function extracts date parts from the payment_date variable.
* NOTE 2: If you get an error that says that the `dimDate` table doesn't exist, then go back to Exercise 1: Step 4 and recreate the tables.

In [ ]:
rows = execute_pg_query(
    conn,
    """
SELECT 
    DISTINCT(TO_CHAR(payment_date :: DATE, 'yyyyMMDD')::integer),
    date(payment_date),
    EXTRACT(year FROM payment_date),
    EXTRACT(quarter FROM payment_date),
    EXTRACT(month FROM payment_date),
    EXTRACT(day FROM payment_date),
    EXTRACT(week FROM payment_date),
    CASE 
        WHEN EXTRACT(ISODOW FROM payment_date) IN (6, 7) 
        THEN true 
        ELSE false 
    END
FROM payment;
    """,
    return_rows=True,
)
main_columns = list(tables["dimDate"].keys())
insert_pg_rows(conn, "dimDate", main_columns, rows)

TODO: Now it's your turn. Populate the `dimCustomer` table with data from the `customer`, `address`, `city`, and `country` tables. Use the starter code as a guide.

In [ ]:
rows = execute_pg_query(
       conn,
       """
SELECT
       nextval('dimcustomer_customer_key_seq'),
       c.customer_id,
       c.first_name,
       c.last_name,
       c.email,
       a.address,
       a.address2,
       a.district,
       ci.city,
       co.country,
       a.postal_code,
       a.phone,
       c.active,
       c.create_date,
       CURRENT_TIMESTAMP,
       CURRENT_TIMESTAMP 
FROM customer c
JOIN address a  ON (c.address_id = a.address_id)
JOIN city ci    ON (a.city_id = ci.city_id)
JOIN country co ON (ci.country_id = co.country_id);
       """,
       return_rows=True,
)
main_columns = list(tables["dimCustomer"].keys())
insert_pg_rows(conn, "dimCustomer", main_columns, rows)

TODO: Populate the `dimMovie` table with data from the `film` and `language` tables. Use the starter code as a guide.

In [ ]:
rows = execute_pg_query(
       conn,
       """
SELECT 
       nextval('dimmovie_movie_key_seq'),
       f.film_id,
       f.title,
       f.description,
       f.release_year,
       l.name,
       orig_lang.name,
       f.rental_duration,
       f.length,
       f.rating,
       f.special_features
FROM film f
JOIN language l              ON (f.language_id=l.language_id)
LEFT JOIN language orig_lang ON (f.original_language_id = orig_lang.language_id);
       """,
       return_rows=True,
)
main_columns = list(tables["dimMovie"].keys())
insert_pg_rows(conn, "dimMovie", main_columns, rows)


TODO: Populate the `dimStore` table with data from the `store`, `staff`, `address`, `city`, and `country` tables. This time, there's no guide. You should write the query from scratch. Use the previous queries as a reference.

In [ ]:
rows = execute_pg_query(
       conn,
       """
SELECT 
       nextval('dimstore_store_key_seq'),
       s.store_id,
       a.address,
       a.address2,
       a.district,
       ci.city,
       co.country,
       a.postal_code,
       e.first_name,
       e.last_name,
       CURRENT_TIMESTAMP,
       CURRENT_TIMESTAMP
FROM store s
JOIN address a  ON (s.address_id = a.address_id)
JOIN city ci    ON (a.city_id = ci.city_id)
JOIN country co ON (ci.country_id = co.country_id)
JOIN staff e ON (s.manager_staff_id = e.staff_id);
       """,
       return_rows=True,
)
main_columns = list(tables["dimStore"].keys())
insert_pg_rows(conn, "dimStore", main_columns, rows)


TODO: Populate the `factSales` table with data from the `payment`, `rental`, and `inventory` tables. This time, there's no guide. You should write the query from scratch. Use the previous queries as a reference.

In [ ]:
rows = execute_pg_query(
       conn,
       """
SELECT 
       DISTINCT(TO_CHAR(payment_date :: DATE, 'yyyyMMDD')::integer),
       p.customer_id,
       i.film_id,
       i.store_id,
       p.amount
FROM payment p
JOIN rental r ON (p.rental_id = r.rental_id)
JOIN inventory i ON (r.inventory_id = i.inventory_id)
JOIN film f ON (i.film_id = f.film_id)
       """,
       return_rows=True,
)
main_columns = list(tables["factSales"].keys())
main_columns.remove("sales_key")  # remove surrogate key
insert_pg_rows(conn, "factSales", main_columns, rows)


In [ ]:
close_pg_connection(conn)